## Example with PRISM Data from lecture

In [1]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import geopandas
from rasterio.mask import mask
import pandas as pd
import numpy as np
import json
import matplotlib.patches as mpatches
import matplotlib.colors
from scipy.signal import medfilt2d

In [2]:
country = geopandas.read_file('/Users/noahgonzales/Desktop/us_states.json')
country = country.to_crs(epsg=4269) 
country = country[(country.NAME=="Iowa")|\
                  (country.NAME=="Illinois")|\
                  (country.NAME=="Nebraska")|\
                  (country.NAME=="Minnesota")|\
                  (country.NAME=="Indiana")|\
                  (country.NAME=="Kansas")|\
                  (country.NAME=="South Dakota")|\
                  (country.NAME=="Ohio")|\
                  (country.NAME=="Missouri")|\
                  (country.NAME=="Wisconsin")]

from rasterstats import zonal_stats
import copy

## Mean Temps for 2013

In [ ]:
zonal_stats0113=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/January/PRISM_tmean_stable_4kmM3_201301_bil.bil')
zonal_stats0113 = pd.DataFrame(zonal_stats0113)
zonal_stats0113.columns = ["temp_"+x for x in zonal_stats0113.columns]
zonal_stats0113.index = country.index
zonal_stats0113 = pd.concat([country,zonal_stats0113],axis=1)

zonal_stats0213=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/February/PRISM_tmean_stable_4kmM3_201302_bil.bil')
zonal_stats0213 = pd.DataFrame(zonal_stats0213)
zonal_stats0213.columns = ["temp_"+x for x in zonal_stats0213.columns]
zonal_stats0213.index = country.index
zonal_stats0213 = pd.concat([country,zonal_stats0213],axis=1)

zonal_stats0313=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/March/PRISM_tmean_stable_4kmM3_201303_bil.bil')
zonal_stats0313 = pd.DataFrame(zonal_stats0313)
zonal_stats0313.columns = ["temp_"+x for x in zonal_stats0313.columns]
zonal_stats0313.index = country.index
zonal_stats0313 = pd.concat([country,zonal_stats0313],axis=1)

zonal_stats0413=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/April/PRISM_tmean_stable_4kmM3_201304_bil.bil')
zonal_stats0413 = pd.DataFrame(zonal_stats0413)
zonal_stats0413.columns = ["temp_"+x for x in zonal_stats0413.columns]
zonal_stats0413.index = country.index
zonal_stats0413 = pd.concat([country,zonal_stats0413],axis=1)

zonal_stats0513=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/May/PRISM_tmean_stable_4kmM3_201305_bil.bil')
zonal_stats0513 = pd.DataFrame(zonal_stats0513)
zonal_stats0513.columns = ["temp_"+x for x in zonal_stats0513.columns]
zonal_stats0513.index = country.index
zonal_stats0513 = pd.concat([country,zonal_stats0513],axis=1)

zonal_stats0613=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/June/PRISM_tmean_stable_4kmM3_201306_bil.bil')
zonal_stats0613 = pd.DataFrame(zonal_stats0613)
zonal_stats0613.columns = ["temp_"+x for x in zonal_stats0613.columns]
zonal_stats0613.index = country.index
zonal_stats0613 = pd.concat([country,zonal_stats0613],axis=1)

zonal_stats0713=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/July/PRISM_tmean_stable_4kmM3_201307_bil.bil')
zonal_stats0713 = pd.DataFrame(zonal_stats0713)
zonal_stats0713.columns = ["temp_"+x for x in zonal_stats0713.columns]
zonal_stats0713.index = country.index
zonal_stats0713 = pd.concat([country,zonal_stats0713],axis=1)

zonal_stats0813=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/August/PRISM_tmean_stable_4kmM3_201308_bil.bil')
zonal_stats0813 = pd.DataFrame(zonal_stats0813)
zonal_stats0813.columns = ["temp_"+x for x in zonal_stats0813.columns]
zonal_stats0813.index = country.index
zonal_stats0813 = pd.concat([country,zonal_stats0813],axis=1)

zonal_stats0913=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/September/PRISM_tmean_stable_4kmM3_201309_bil.bil')
zonal_stats0913 = pd.DataFrame(zonal_stats0913)
zonal_stats0913.columns = ["temp_"+x for x in zonal_stats0913.columns]
zonal_stats0913.index = country.index
zonal_stats0913 = pd.concat([country,zonal_stats0813],axis=1)

zonal_stats1013=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/October/PRISM_tmean_stable_4kmM3_201310_bil.bil')
zonal_stats1013 = pd.DataFrame(zonal_stats1013)
zonal_stats1013.columns = ["temp_"+x for x in zonal_stats1013.columns]
zonal_stats1013.index = country.index
zonal_stats1013 = pd.concat([country,zonal_stats1013],axis=1)

zonal_stats1113=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/November/PRISM_tmean_stable_4kmM3_201311_bil.bil')
zonal_stats1113 = pd.DataFrame(zonal_stats1113)
zonal_stats1113.columns = ["temp_"+x for x in zonal_stats1113.columns]
zonal_stats1113.index = country.index
zonal_stats1113 = pd.concat([country,zonal_stats1113],axis=1)

zonal_stats1213=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/December/PRISM_tmean_stable_4kmM3_201312_bil.bil')
zonal_stats1213 = pd.DataFrame(zonal_stats1213)
zonal_stats1213.columns = ["temp_"+x for x in zonal_stats1213.columns]
zonal_stats1213.index = country.index
zonal_stats1213 = pd.concat([country,zonal_stats1213],axis=1)


## Mean Temps for 2014

In [ ]:
zonal_stats0113=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/January/PRISM_tmean_stable_4kmM3_201301_bil.bil')
zonal_stats0113 = pd.DataFrame(zonal_stats0113)
zonal_stats0113.columns = ["temp_"+x for x in zonal_stats0113.columns]
zonal_stats0113.index = country.index
zonal_stats0113 = pd.concat([country,zonal_stats0113],axis=1)

zonal_stats0213=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/February/PRISM_tmean_stable_4kmM3_201302_bil.bil')
zonal_stats0213 = pd.DataFrame(zonal_stats0213)
zonal_stats0213.columns = ["temp_"+x for x in zonal_stats0213.columns]
zonal_stats0213.index = country.index
zonal_stats0213 = pd.concat([country,zonal_stats0213],axis=1)

zonal_stats0313=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/March/PRISM_tmean_stable_4kmM3_201303_bil.bil')
zonal_stats0313 = pd.DataFrame(zonal_stats0313)
zonal_stats0313.columns = ["temp_"+x for x in zonal_stats0313.columns]
zonal_stats0313.index = country.index
zonal_stats0313 = pd.concat([country,zonal_stats0313],axis=1)


zonal_stats0414=zonal_stats(country, '/Users/noahgonzales/Desktop/2014 Temp/April/PRISM_tmean_stable_4kmM3_201404_bil.bil')
zonal_stats0414 = pd.DataFrame(zonal_stats0414)
zonal_stats0414.columns = ["temp_"+x for x in zonal_stats0414.columns]
zonal_stats0414.index = country.index
zonal_stats0414 = pd.concat([country,zonal_stats0414],axis=1)

zonal_stats0514=zonal_stats(country, '/Users/noahgonzales/Desktop/2014 Temp/May/PRISM_tmean_stable_4kmM3_201405_bil.bil')
zonal_stats0514 = pd.DataFrame(zonal_stats0514)
zonal_stats0514.columns = ["temp_"+x for x in zonal_stats0514.columns]
zonal_stats0514.index = country.index
zonal_stats0514 = pd.concat([country,zonal_stats0514],axis=1)

zonal_stats0614=zonal_stats(country, '/Users/noahgonzales/Desktop/2014 Temp/June/PRISM_tmean_stable_4kmM3_201406_bil.bil')
zonal_stats0614 = pd.DataFrame(zonal_stats0614)
zonal_stats0614.columns = ["temp_"+x for x in zonal_stats0614.columns]
zonal_stats0614.index = country.index
zonal_stats0614 = pd.concat([country,zonal_stats0614],axis=1)

zonal_stats0714=zonal_stats(country, '/Users/noahgonzales/Desktop/2014 Temp/July/PRISM_tmean_stable_4kmM3_201407_bil.bil')
zonal_stats0714 = pd.DataFrame(zonal_stats0714)
zonal_stats0714.columns = ["temp_"+x for x in zonal_stats0714.columns]
zonal_stats0714.index = country.index
zonal_stats0714 = pd.concat([country,zonal_stats0714],axis=1)

zonal_stats0814=zonal_stats(country, '/Users/noahgonzales/Desktop/2014 Temp/August/PRISM_tmean_stable_4kmM3_201408_bil.bil')
zonal_stats0814 = pd.DataFrame(zonal_stats0814)
zonal_stats0814.columns = ["temp_"+x for x in zonal_stats0814.columns]
zonal_stats0814.index = country.index
zonal_stats0814 = pd.concat([country,zonal_stats0814],axis=1)



zonal_stats0913=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/September/PRISM_tmean_stable_4kmM3_201309_bil.bil')
zonal_stats0913 = pd.DataFrame(zonal_stats0913)
zonal_stats0913.columns = ["temp_"+x for x in zonal_stats0913.columns]
zonal_stats0913.index = country.index
zonal_stats0913 = pd.concat([country,zonal_stats0813],axis=1)


zonal_stats1013=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/October/PRISM_tmean_stable_4kmM3_201310_bil.bil')
zonal_stats1013 = pd.DataFrame(zonal_stats1013)
zonal_stats1013.columns = ["temp_"+x for x in zonal_stats1013.columns]
zonal_stats1013.index = country.index
zonal_stats1013 = pd.concat([country,zonal_stats1013],axis=1)


zonal_stats1113=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/November/PRISM_tmean_stable_4kmM3_201311_bil.bil')
zonal_stats1113 = pd.DataFrame(zonal_stats1113)
zonal_stats1113.columns = ["temp_"+x for x in zonal_stats1113.columns]
zonal_stats1113.index = country.index
zonal_stats1113 = pd.concat([country,zonal_stats1113],axis=1)


zonal_stats1213=zonal_stats(country, '/Users/noahgonzales/Desktop/2013 Temp/December/PRISM_tmean_stable_4kmM3_201312_bil.bil')
zonal_stats1213 = pd.DataFrame(zonal_stats1213)
zonal_stats1213.columns = ["temp_"+x for x in zonal_stats1213.columns]
zonal_stats1213.index = country.index
zonal_stats1213 = pd.concat([country,zonal_stats1213],axis=1)


In [ ]:
zonal_stats0414['month']=4
zonal_stats0514['month']=5
zonal_stats0614['month']=6
zonal_stats0714['month']=7
zonal_stats0814['month']=8

In [ ]:
zonal_stats14=copy.deepcopy(zonal_stats0414)  
zonal_stats14=zonal_stats14.append([zonal_stats0514,zonal_stats0614,zonal_stats0714,zonal_stats0814],ignore_index=True)
zonal_stats14

## Mean Temps 2015

In [ ]:
zonal_stats0115=zonal_stats(country, '/Users/noahgonzales/Desktop/2015 Temp/January/PRISM_tmean_stable_4kmM3_201501_bil.bil')
zonal_stats0115 = pd.DataFrame(zonal_stats0115)
zonal_stats0115.columns = ["temp_"+x for x in zonal_stats0115.columns]
zonal_stats0115.index = country.index
zonal_stats0115 = pd.concat([country,zonal_stats0115],axis=1)

zonal_stats0215=zonal_stats(country, '/Users/noahgonzales/Desktop/2015 Temp/February/PRISM_tmean_stable_4kmM3_201502_bil.bil')
zonal_stats0215 = pd.DataFrame(zonal_stats0215)
zonal_stats0215.columns = ["temp_"+x for x in zonal_stats0215.columns]
zonal_stats0215.index = country.index
zonal_stats0215 = pd.concat([country,zonal_stats0215],axis=1)

zonal_stats0315=zonal_stats(country, '/Users/noahgonzales/Desktop/2015 Temp/March/PRISM_tmean_stable_4kmM3_201503_bil.bil')
zonal_stats0315 = pd.DataFrame(zonal_stats0315)
zonal_stats0315.columns = ["temp_"+x for x in zonal_stats0315.columns]
zonal_stats0315.index = country.index
zonal_stats0315 = pd.concat([country,zonal_stats0315],axis=1)

In [ ]:
zonal_stats0415=zonal_stats(country, '/Users/noahgonzales/Desktop/2015 Temp/April/PRISM_tmean_stable_4kmM3_201504_bil.bil')
zonal_stats0415 = pd.DataFrame(zonal_stats0415)
zonal_stats0415.columns = ["temp_"+x for x in zonal_stats0415.columns]
zonal_stats0415.index = country.index
zonal_stats0415 = pd.concat([country,zonal_stats0415],axis=1)

zonal_stats0515=zonal_stats(country, '/Users/noahgonzales/Desktop/2015 Temp/May/PRISM_tmean_stable_4kmM3_201505_bil.bil')
zonal_stats0515 = pd.DataFrame(zonal_stats0515)
zonal_stats0515.columns = ["temp_"+x for x in zonal_stats0515.columns]
zonal_stats0515.index = country.index
zonal_stats0515 = pd.concat([country,zonal_stats0515],axis=1)

zonal_stats0615=zonal_stats(country, '/Users/noahgonzales/Desktop/2015 Temp/June/PRISM_tmean_stable_4kmM3_201506_bil.bil')
zonal_stats0615 = pd.DataFrame(zonal_stats0615)
zonal_stats0615.columns = ["temp_"+x for x in zonal_stats0615.columns]
zonal_stats0615.index = country.index
zonal_stats0615 = pd.concat([country,zonal_stats0615],axis=1)

zonal_stats0715=zonal_stats(country, '/Users/noahgonzales/Desktop/2015 Temp/July/PRISM_tmean_stable_4kmM3_201507_bil.bil')
zonal_stats0715 = pd.DataFrame(zonal_stats0715)
zonal_stats0715.columns = ["temp_"+x for x in zonal_stats0715.columns]
zonal_stats0715.index = country.index
zonal_stats0715 = pd.concat([country,zonal_stats0715],axis=1)

zonal_stats0815=zonal_stats(country, '/Users/noahgonzales/Desktop/2015 Temp/August/PRISM_tmean_stable_4kmM3_201508_bil.bil')
zonal_stats0815 = pd.DataFrame(zonal_stats0815)
zonal_stats0815.columns = ["temp_"+x for x in zonal_stats0815.columns]
zonal_stats0815.index = country.index
zonal_stats0815 = pd.concat([country,zonal_stats0815],axis=1)

zonal_stats0915=zonal_stats(country, '/Users/noahgonzales/Desktop/2015 Temp/September/PRISM_tmean_stable_4kmM3_201509_bil.bil')
zonal_stats0915 = pd.DataFrame(zonal_stats0915)
zonal_stats0915.columns = ["temp_"+x for x in zonal_stats0915.columns]
zonal_stats0915.index = country.index
zonal_stats0915 = pd.concat([country,zonal_stats0915],axis=1)

zonal_stats1015=zonal_stats(country, '/Users/noahgonzales/Desktop/2015 Temp/October/PRISM_tmean_stable_4kmM3_201510_bil.bil')
zonal_stats1015 = pd.DataFrame(zonal_stats1015)
zonal_stats1015.columns = ["temp_"+x for x in zonal_stats1015.columns]
zonal_stats1015.index = country.index
zonal_stats1015 = pd.concat([country,zonal_stats1015],axis=1)



In [ ]:

zonal_stats1115=zonal_stats(country, '/Users/noahgonzales/Desktop/2015 Temp/November/PRISM_tmean_stable_4kmM3_201511_bil.bil')
zonal_stats1115 = pd.DataFrame(zonal_stats1115)
zonal_stats1115.columns = ["temp_"+x for x in zonal_stats1115.columns]
zonal_stats1115.index = country.index
zonal_stats1115 = pd.concat([country,zonal_stats1115],axis=1)

zonal_stats1215=zonal_stats(country, '/Users/noahgonzales/Desktop/2015 Temp/December/PRISM_tmean_stable_4kmM3_201512_bil.bil')
zonal_stats1215 = pd.DataFrame(zonal_stats1215)
zonal_stats1215.columns = ["temp_"+x for x in zonal_stats1215.columns]
zonal_stats1215.index = country.index
zonal_stats1215 = pd.concat([country,zonal_stats1115],axis=1)


In [ ]:
zonal_stats0415['month']=4
zonal_stats0515['month']=5
zonal_stats0615['month']=6
zonal_stats0715['month']=7
zonal_stats0815['month']=8
zonal_stats0915['month']=9
zonal_stats1015['month']=10
zonal_stats1115['month']=11


In [ ]:
zonal_stats15=copy.deepcopy(zonal_stats0415)  
zonal_stats15=zonal_stats15.append([zonal_stats0515,zonal_stats0615,zonal_stats0715,zonal_stats0815,zonal_stats0915,zonal_stats1015,zonal_stats1115,],ignore_index=True)
zonal_stats15

## Mean Temps 2016

In [ ]:
zonal_stats0416=zonal_stats(country, '/Users/noahgonzales/Desktop/2016 Temp/April/PRISM_tmean_stable_4kmM3_201604_bil.bil')
zonal_stats0416 = pd.DataFrame(zonal_stats0416)
zonal_stats0416.columns = ["temp_"+x for x in zonal_stats0416.columns]
zonal_stats0416.index = country.index
zonal_stats0416 = pd.concat([country,zonal_stats0416],axis=1)

zonal_stats0516=zonal_stats(country, '/Users/noahgonzales/Desktop/2016 Temp/May/PRISM_tmean_stable_4kmM3_201605_bil.bil')
zonal_stats0516 = pd.DataFrame(zonal_stats0516)
zonal_stats0516.columns = ["temp_"+x for x in zonal_stats0516.columns]
zonal_stats0516.index = country.index
zonal_stats0516 = pd.concat([country,zonal_stats0516],axis=1)

zonal_stats0616=zonal_stats(country, '/Users/noahgonzales/Desktop/2016 Temp/June/PRISM_tmean_stable_4kmM3_201606_bil.bil')
zonal_stats0616 = pd.DataFrame(zonal_stats0616)
zonal_stats0616.columns = ["temp_"+x for x in zonal_stats0616.columns]
zonal_stats0616.index = country.index
zonal_stats0616 = pd.concat([country,zonal_stats0616],axis=1)

zonal_stats0716=zonal_stats(country, '/Users/noahgonzales/Desktop/2016 Temp/July/PRISM_tmean_stable_4kmM3_201607_bil.bil')
zonal_stats0716 = pd.DataFrame(zonal_stats0716)
zonal_stats0716.columns = ["temp_"+x for x in zonal_stats0716.columns]
zonal_stats0716.index = country.index
zonal_stats0716 = pd.concat([country,zonal_stats0716],axis=1)

zonal_stats0816=zonal_stats(country, '/Users/noahgonzales/Desktop/2016 Temp/August/PRISM_tmean_stable_4kmM3_201608_bil.bil')
zonal_stats0816 = pd.DataFrame(zonal_stats0816)
zonal_stats0816.columns = ["temp_"+x for x in zonal_stats0816.columns]
zonal_stats0816.index = country.index
zonal_stats0816 = pd.concat([country,zonal_stats0816],axis=1)

In [ ]:
zonal_stats0416['month']=4
zonal_stats0516['month']=5
zonal_stats0616['month']=6
zonal_stats0716['month']=7
zonal_stats0816['month']=8

In [ ]:
zonal_stats16=copy.deepcopy(zonal_stats0416)  
zonal_stats16=zonal_stats16.append([zonal_stats0516,zonal_stats0616,zonal_stats0716,zonal_stats0816],ignore_index=True)
zonal_stats16

## Mean Temps 2017

In [ ]:
zonal_stats0417=zonal_stats(country, '/Users/noahgonzales/Desktop/2017 Temp/April/PRISM_tmean_stable_4kmM3_201704_bil.bil')
zonal_stats0417 = pd.DataFrame(zonal_stats0417)
zonal_stats0417.columns = ["temp_"+x for x in zonal_stats0417.columns]
zonal_stats0417.index = country.index
zonal_stats0417 = pd.concat([country,zonal_stats0417],axis=1)

zonal_stats0517=zonal_stats(country, '/Users/noahgonzales/Desktop/2017 Temp/May/PRISM_tmean_stable_4kmM3_201705_bil.bil')
zonal_stats0517 = pd.DataFrame(zonal_stats0517)
zonal_stats0517.columns = ["temp_"+x for x in zonal_stats0517.columns]
zonal_stats0517.index = country.index
zonal_stats0517 = pd.concat([country,zonal_stats0517],axis=1)

zonal_stats0617=zonal_stats(country, '/Users/noahgonzales/Desktop/2017 Temp/June/PRISM_tmean_stable_4kmM3_201706_bil.bil')
zonal_stats0617 = pd.DataFrame(zonal_stats0617)
zonal_stats0617.columns = ["temp_"+x for x in zonal_stats0617.columns]
zonal_stats0617.index = country.index
zonal_stats0617 = pd.concat([country,zonal_stats0617],axis=1)

zonal_stats0717=zonal_stats(country, '/Users/noahgonzales/Desktop/2017 Temp/July/PRISM_tmean_stable_4kmM3_201707_bil.bil')
zonal_stats0717 = pd.DataFrame(zonal_stats0717)
zonal_stats0717.columns = ["temp_"+x for x in zonal_stats0717.columns]
zonal_stats0717.index = country.index
zonal_stats0717 = pd.concat([country,zonal_stats0717],axis=1)

zonal_stats0817=zonal_stats(country, '/Users/noahgonzales/Desktop/2017 Temp/August/PRISM_tmean_stable_4kmM3_201708_bil.bil')
zonal_stats0817 = pd.DataFrame(zonal_stats0817)
zonal_stats0817.columns = ["temp_"+x for x in zonal_stats0817.columns]
zonal_stats0817.index = country.index
zonal_stats0817 = pd.concat([country,zonal_stats0817],axis=1)

In [ ]:
zonal_stats0417['month']=4
zonal_stats0517['month']=5
zonal_stats0617['month']=6
zonal_stats0717['month']=7
zonal_stats0817['month']=8

In [ ]:
zonal_stats17=copy.deepcopy(zonal_stats0417)  
zonal_stats17=zonal_stats17.append([zonal_stats0517,zonal_stats0617,zonal_stats0717,zonal_stats0817],ignore_index=True)
zonal_stats17

## Mean Temps 2018

In [ ]:
zonal_stats0418=zonal_stats(country, '/Users/noahgonzales/Desktop/2018 Temp/April/PRISM_tmean_stable_4kmM3_201804_bil.bil')
zonal_stats0418 = pd.DataFrame(zonal_stats0418)
zonal_stats0418.columns = ["temp_"+x for x in zonal_stats0418.columns]
zonal_stats0418.index = country.index
zonal_stats0418 = pd.concat([country,zonal_stats0418],axis=1)

zonal_stats0518=zonal_stats(country, '/Users/noahgonzales/Desktop/2018 Temp/May/PRISM_tmean_stable_4kmM3_201805_bil.bil')
zonal_stats0518 = pd.DataFrame(zonal_stats0518)
zonal_stats0518.columns = ["temp_"+x for x in zonal_stats0518.columns]
zonal_stats0518.index = country.index
zonal_stats0518 = pd.concat([country,zonal_stats0518],axis=1)

zonal_stats0618=zonal_stats(country, '/Users/noahgonzales/Desktop/2018 Temp/June/PRISM_tmean_stable_4kmM3_201806_bil.bil')
zonal_stats0618 = pd.DataFrame(zonal_stats0618)
zonal_stats0618.columns = ["temp_"+x for x in zonal_stats0618.columns]
zonal_stats0618.index = country.index
zonal_stats0618 = pd.concat([country,zonal_stats0618],axis=1)

zonal_stats0718=zonal_stats(country, '/Users/noahgonzales/Desktop/2018 Temp/July/PRISM_tmean_stable_4kmM3_201807_bil.bil')
zonal_stats0718 = pd.DataFrame(zonal_stats0718)
zonal_stats0718.columns = ["temp_"+x for x in zonal_stats0718.columns]
zonal_stats0718.index = country.index
zonal_stats0718 = pd.concat([country,zonal_stats0718],axis=1)

zonal_stats0818=zonal_stats(country, '/Users/noahgonzales/Desktop/2018 Temp/August/PRISM_tmean_stable_4kmM3_201808_bil.bil')
zonal_stats0818 = pd.DataFrame(zonal_stats0818)
zonal_stats0818.columns = ["temp_"+x for x in zonal_stats0818.columns]
zonal_stats0818.index = country.index
zonal_stats0818 = pd.concat([country,zonal_stats0818],axis=1)

In [ ]:
zonal_stats0418['month']=4
zonal_stats0518['month']=5
zonal_stats0618['month']=6
zonal_stats0718['month']=7
zonal_stats0818['month']=8

In [ ]:
zonal_stats18=copy.deepcopy(zonal_stats0418)  
zonal_stats18=zonal_stats18.append([zonal_stats0518,zonal_stats0618,zonal_stats0718,zonal_stats0818],ignore_index=True)
zonal_stats18

## Mean 2019

In [ ]:
zonal_stats0419=zonal_stats(country, '/Users/noahgonzales/Desktop/2019 Temp/April/PRISM_tmean_stable_4kmM3_201904_bil.bil')
zonal_stats0419 = pd.DataFrame(zonal_stats0419)
zonal_stats0419.columns = ["temp_"+x for x in zonal_stats0419.columns]
zonal_stats0419.index = country.index
zonal_stats0419 = pd.concat([country,zonal_stats0419],axis=1)

zonal_stats0519=zonal_stats(country, '/Users/noahgonzales/Desktop/2019 Temp/May/PRISM_tmean_stable_4kmM3_201905_bil.bil')
zonal_stats0519 = pd.DataFrame(zonal_stats0519)
zonal_stats0519.columns = ["temp_"+x for x in zonal_stats0519.columns]
zonal_stats0519.index = country.index
zonal_stats0519 = pd.concat([country,zonal_stats0519],axis=1)

zonal_stats0619=zonal_stats(country, '/Users/noahgonzales/Desktop/2019 Temp/June/PRISM_tmean_stable_4kmM3_201906_bil.bil')
zonal_stats0619 = pd.DataFrame(zonal_stats0619)
zonal_stats0619.columns = ["temp_"+x for x in zonal_stats0619.columns]
zonal_stats0619.index = country.index
zonal_stats0619 = pd.concat([country,zonal_stats0619],axis=1)

zonal_stats0719=zonal_stats(country, '/Users/noahgonzales/Desktop/2019 Temp/July/PRISM_tmean_stable_4kmM3_201907_bil.bil')
zonal_stats0719 = pd.DataFrame(zonal_stats0719)
zonal_stats0719.columns = ["temp_"+x for x in zonal_stats0719.columns]
zonal_stats0719.index = country.index
zonal_stats0719 = pd.concat([country,zonal_stats0719],axis=1)

zonal_stats0819=zonal_stats(country, '/Users/noahgonzales/Desktop/2019 Temp/August/PRISM_tmean_stable_4kmM3_201908_bil.bil')
zonal_stats0819 = pd.DataFrame(zonal_stats0819)
zonal_stats0819.columns = ["temp_"+x for x in zonal_stats0819.columns]
zonal_stats0819.index = country.index
zonal_stats0819 = pd.concat([country,zonal_stats0819],axis=1)

In [ ]:
zonal_stats0419['month']=4
zonal_stats0519['month']=5
zonal_stats0619['month']=6
zonal_stats0719['month']=7
zonal_stats0819['month']=8

In [ ]:
zonal_stats19=copy.deepcopy(zonal_stats0418)  
zonal_stats19=zonal_stats19.append([zonal_stats0519,zonal_stats0619,zonal_stats0719,zonal_stats0819],ignore_index=True)
zonal_stats19